In [5]:
import requests
import mysql.connector
import re

In [6]:
db_cnx = mysql.connector.connect(user="docker", password="docker", host="localhost", database="tcc")

# Salvando todos os deputados ativos atualmente

In [ ]:
listagem_deputados_url = "https://dadosabertos.camara.leg.br/api/v2/deputados?itens=100"
hasNext = True
cursor = db_cnx.cursor()

while hasNext:
    response = requests.get(listagem_deputados_url).json()
    for deputado in response["dados"]:
        statement = """
            insert into deputado (id, nome, nome_eleitoral, situacao,
                                  condicao_eleitoral, partido, estado, uri_foto)
            values ( %(id)s, %(nome)s, %(nome_eleitoral)s, %(situacao)s,
             %(condicao_eleitoral)s,
             %(partido)s,
             %(estado)s,
             %(uri_foto)s
             );
        """

        params = {
            "id": deputado["id"],
            "nome_eleitoral": deputado["nome"],
            "estado": deputado["siglaUf"],
            "partido": deputado["siglaPartido"],
            "uri_foto": deputado.get("uriFoto")
        }

        detalhes = requests.get(deputado["uri"]).json()["dados"]

        params["nome"] = detalhes["nomeCivil"]
        params["situacao"] = detalhes["ultimoStatus"]["situacao"]
        params["condicao_eleitoral"] = detalhes["ultimoStatus"]["condicaoEleitoral"]

        try:
            cursor.execute(statement, params)
        except Exception as e:
            cursor.close()
            print(params)
            raise e

    hasNext = False
    for i in response["links"]:
        if i.get("rel") == "next":
            hasNext = True
            listagem_deputados_url = i["href"]
            break

db_cnx.commit()

In [ ]:
db_cnx.close()